In [ ]:
# install huggingface transformers module

# !git clone https://github.com/huggingface/transformers.git
# !pip install -U ./transformers

In [12]:
import pandas as pd
import numpy as np
import torch
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding 
from transformers import BertTokenizer, BertModel, BertConfig
from torch import cuda


In [3]:
# define customized evaluation metric

def multi_label_metrics(predictions, labels, threshold=0.5):

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)

    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    
    return result

In [4]:
# dataset class wrapper as fine-tuning input

class ReutersDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.labels = self.data.label_list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float)
        }

In [5]:
# parameters

MAX_LEN = 512
batch_size = 4
learning_rate = 2e-5

In [6]:
# load and process cleaned data

clean_reuters_data = pd.read_pickle('clean_reuters_data.pkl')

flattened_topics = pd.DataFrame(clean_reuters_data.topics.sum(), columns=['topic'])
unique_topics = flattened_topics.topic.unique().tolist()

id2topic = {idx:topic for idx, topic in enumerate(unique_topics)}
topic2id = {topic:idx for idx, topic in enumerate(unique_topics)}

def get_mapping(x):
    res = [0] * len(unique_topics)
    for key in x:
        res[topic2id[key]] = 1
    
    return res

clean_reuters_data['label_list'] = clean_reuters_data.topics.apply(lambda x: get_mapping(x))

train_dataset = clean_reuters_data[clean_reuters_data.lewis_split == 'TRAIN'][['text', 'label_list']].reset_index(drop=True)
test_dataset = clean_reuters_data[clean_reuters_data.lewis_split == 'TEST'][['text', 'label_list']].reset_index(drop=True)


# Buildin Functionality

In [ ]:
# initialize bert pre-trained model

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(unique_topics))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# generate dataset

training_set = ReutersDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = ReutersDataset(test_dataset, tokenizer, MAX_LEN)

In [ ]:
# generate traing arguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [ ]:
# train the model!

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=testing_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
# show evaluation results

trainer.evaluate()

# Design your own layers

In [7]:
EPOCHS = 1

device = 'cuda' if cuda.is_available() else 'cpu'

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [10]:
# get the training loader

train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

training_set = ReutersDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = ReutersDataset(test_dataset, tokenizer, MAX_LEN)
training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [13]:
# Add layers on top of pre-trained BERT model

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear(768, 120)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1= self.l1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1[1])
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [14]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)


def train(epoch):
    model.train()
    for _, data in enumerate(training_loader, 0):
        input_ids = data['input_ids'].to(device, dtype = torch.long)
        attention_mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        labels = data['labels'].to(device, dtype = torch.float)

        outputs = model(input_ids, attention_mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, labels)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

def validation(epoch):
    
    model.eval()
    final_targets=[]
    final_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            input_ids = data['input_ids'].to(device, dtype = torch.long)
            attention_mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            labels = data['labels'].to(device, dtype = torch.float)
            
            outputs = model(input_ids, attention_mask, token_type_ids)
            final_targets.extend(labels.cpu().detach().numpy().tolist())
            final_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            
    return final_outputs, final_targets

In [17]:
# Train the model

for epoch in range(EPOCHS):
    train(epoch)

D:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.6788820028305054


In [24]:
# Assess Performance

for epoch in range(EPOCHS):
    
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = accuracy_score(targets, outputs)
    f1_micro_average = f1_score(targets, outputs, average='micro')
    roc_auc = roc_auc_score(targets, outputs, average='micro')
    
    print(f1_micro_average, roc_auc, accuracy)

0.6681948424068768 0.7546525895415935 0.5886190648785792
